In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
import sys
import os
import time
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision.models import vgg
import argparse
sys.path.insert(0, '/content/3D-BoundingBox')
!git clone 'https://github.com/skhadem/3D-BoundingBox.git'
!unzip '/content/gdrive/MyDrive/image_2.zip'
!cp '/content/gdrive/MyDrive/yolov3.weights' '/content/3D-BoundingBox/weights'
!cp '/content/gdrive/MyDrive/epoch_10.pkl' '/content/3D-BoundingBox/weights'
!cp '/content/image_2' '/content/3D-BoundingBox' -r
!cp '/content/gdrive/MyDrive/calib_cam_to_cam.txt' '/content/3D-BoundingBox'
from torch_lib.Dataset import *
from library.Math import *
from library.Plotting import *
from torch_lib import Model, ClassAverages
from yolo.yolo import cv_Yolo
from google.colab.patches import cv2_imshow

In [ ]:
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

In [ ]:
parser = argparse.ArgumentParser()

In [ ]:
def plot_regressed_3d_bbox(img, cam_to_img, box_2d, dimensions, alpha, theta_ray, img_2d=None):

    # the math! returns X, the corners used for constraint
    location, X = calc_location(dimensions, cam_to_img, box_2d, alpha, theta_ray)

    orient = alpha + theta_ray

    if img_2d is not None:
        plot_2d_box(img_2d, box_2d)

    plot_3d_box(img, cam_to_img, orient, dimensions, location) # 3d boxes

    return location

In [ ]:
weights_path = '/content/3D-BoundingBox/weights'
model_lst = [x for x in sorted(os.listdir(weights_path)) if x.endswith('.pkl')]
if len(model_lst) == 0:
    print('No previous model found, please train first!')
    exit()
else:
    print('Using previous model %s'%model_lst[-1])
    my_vgg = vgg.vgg19_bn(pretrained=True)
    # TODO: load bins from file or something
    model = Model.Model(features=my_vgg.features, bins=2).cuda()
    checkpoint = torch.load(weights_path + '/%s'%model_lst[-1])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

Using previous model epoch_10.pkl


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# load yolo
yolo_path = '/content/3D-BoundingBox/weights'
yolo = cv_Yolo(yolo_path)

In [ ]:
# TODO: clean up how this is done. flag?
angle_bins = generate_bins(2)

img_path = '/content/3D-BoundingBox/image_2/'
print(img_path)
calib_file = '/content/3D-BoundingBox/calib_cam_to_cam.txt'
print(calib_file)

/content/3D-BoundingBox/image_2/
/content/3D-BoundingBox/calib_cam_to_cam.txt


In [ ]:
averages = ClassAverages.ClassAverages()

In [ ]:
try:
    ids = [x.split('.')[0] for x in sorted(os.listdir(img_path))]
except:
    print("\nError: no images in %s"%img_path)
    exit()

In [ ]:
for img_id in ids:

        start_time = time.time()

        img_file = img_path + img_id + ".png"
        print(img_file)

        truth_img = cv2.imread(img_file)
        img = np.copy(truth_img)
        yolo_img = np.copy(truth_img)

        detections = yolo.detect(yolo_img)

        for detection in detections:

            if not averages.recognized_class(detection.detected_class):
                continue

            try:
                detectedObject = DetectedObject(img, detection.detected_class, detection.box_2d, calib_file)
            except:
                continue

            theta_ray = detectedObject.theta_ray
            input_img = detectedObject.img
            proj_matrix = detectedObject.proj_matrix
            box_2d = detection.box_2d
            detected_class = detection.detected_class

            input_tensor = torch.zeros([1,3,224,224]).cuda()
            input_tensor[0,:,:,:] = input_img

            [orient, conf, dim] = model(input_tensor)
            orient = orient.cpu().data.numpy()[0, :, :]
            conf = conf.cpu().data.numpy()[0, :]
            dim = dim.cpu().data.numpy()[0, :]

            dim += averages.get_item(detected_class)

            argmax = np.argmax(conf)
            orient = orient[argmax, :]
            cos = orient[0]
            sin = orient[1]
            alpha = np.arctan2(sin, cos)
            alpha += angle_bins[argmax]
            alpha -= np.pi


            location = plot_regressed_3d_bbox(img, proj_matrix, box_2d, dim, alpha, theta_ray, truth_img)

        cv2_imshow(img)
        
        cv2.imwrite(img_id + ".jpg", img)

In [ ]:
import os
images = []


for i in range(1,446+1):
  n = len(str(i))
  identifier = (3-n)*"0" + str(i)
  # print(identifier)

  img_fname = "/content/0000000"+identifier+".jpg"
  img = cv2.imread(img_fname)
  # img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  images.append(img)

In [ ]:
images

[array([[[255, 255, 173],
         [255, 255, 173],
         [255, 255, 172],
         ...,
         [ 64,  53,  31],
         [ 62,  51,  31],
         [ 60,  51,  31]],
 
        [[255, 255, 172],
         [255, 255, 172],
         [255, 255, 171],
         ...,
         [ 61,  50,  30],
         [ 58,  48,  30],
         [ 57,  47,  29]],
 
        [[255, 254, 173],
         [255, 254, 173],
         [255, 254, 173],
         ...,
         [ 53,  40,  24],
         [ 53,  42,  28],
         [ 51,  40,  26]],
 
        ...,
 
        [[162, 167, 192],
         [166, 173, 193],
         [167, 175, 188],
         ...,
         [  4,   1,   3],
         [  2,   2,   2],
         [  2,   3,   1]],
 
        [[167, 174, 189],
         [172, 178, 189],
         [176, 182, 187],
         ...,
         [  4,   1,   3],
         [  3,   3,   3],
         [  3,   4,   2]],
 
        [[165, 168, 166],
         [168, 169, 165],
         [173, 172, 162],
         ...,
         [  3,   2,   4],
  

In [ ]:
import cv2
# cap = cv2.VideoCapture('highway.mp4')
# ret, frame = cap.read()
frame_height, frame_width, _ = images[0].shape
out = cv2.VideoWriter('output2.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
print("Processing Video...")
for frame in images:
  out.write(frame)
out.release()
print("Done processing video")

Processing Video...
Done processing video


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive
